In [2]:
import numpy as np
import torch
import torch.optim as optim
torch.set_printoptions(edgeitems=2, linewidth=75)

In [3]:
t_c = torch.tensor([0.5, 14.0, 15.0, 28.0, 11.0,
                    8.0, 3.0, -4.0, 6.0, 13.0, 21.0])
t_u = torch.tensor([35.7, 55.9, 58.2, 81.9, 56.3, 48.9,
                    33.9, 21.8, 48.4, 60.4, 68.4])
t_un = 0.1 * t_u

In [5]:
def model(t_u, w1, w2, b):
    return w2 * t_u ** 2 + w1 * t_u + b

In [6]:
def loss_fn(t_p, t_c):
    squared_diffs = (t_p - t_c)**2
    return squared_diffs.mean()

In [25]:
n_samples = t_u.shape[0]
n_val = int(0.2 * n_samples)

shuffled_indices = torch.randperm(n_samples)

train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

train_indices, val_indices

(tensor([0, 7, 4, 2, 5, 1, 3, 8, 6]), tensor([ 9, 10]))

In [8]:
train_t_u = t_u[train_indices]
train_t_c = t_c[train_indices]

val_t_u = t_u[val_indices]
val_t_c = t_c[val_indices]

train_t_un = 0.1 * train_t_u
val_t_un = 0.1 * val_t_u

In [9]:
def training_loop(n_epochs, optimizer, params, train_t_u, val_t_u,
                  train_t_c, val_t_c):
    for epoch in range(1, n_epochs + 1):
        train_t_p = model(train_t_u, *params)
        train_loss = loss_fn(train_t_p, train_t_c)
                             
        val_t_p = model(val_t_u, *params)
        val_loss = loss_fn(val_t_p, val_t_c)
        
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        if epoch <= 3 or epoch % 500 == 0:
            print(f"Epoch {epoch}, Training loss {train_loss.item():.4f},"
                  f" Validation loss {val_loss.item():.4f}")
            
    return params

In [26]:
params = torch.tensor([1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-1
optimizer = optim.Adam([params], lr=learning_rate)

training_loop(
    n_epochs = 8000, 
    optimizer = optimizer,
    params = params,
    train_t_u = train_t_un,
    val_t_u = val_t_un, 
    train_t_c = train_t_c,
    val_t_c = val_t_c)

Epoch 1, Training loss 764.6102, Validation loss 276.1225
Epoch 2, Training loss 546.1442, Validation loss 210.2112
Epoch 3, Training loss 365.9996, Validation loss 154.2220
Epoch 500, Training loss 2.8550, Validation loss 4.0979
Epoch 1000, Training loss 2.7955, Validation loss 3.3896
Epoch 1500, Training loss 2.7271, Validation loss 2.5719
Epoch 2000, Training loss 2.6621, Validation loss 1.7761
Epoch 2500, Training loss 2.6100, Validation loss 1.1103
Epoch 3000, Training loss 2.5750, Validation loss 0.6324
Epoch 3500, Training loss 2.5561, Validation loss 0.3408
Epoch 4000, Training loss 2.5482, Validation loss 0.1902
Epoch 4500, Training loss 2.5459, Validation loss 0.1236
Epoch 5000, Training loss 2.5454, Validation loss 0.0980
Epoch 5500, Training loss 2.5454, Validation loss 0.0901
Epoch 6000, Training loss 2.5454, Validation loss 0.0876
Epoch 6500, Training loss 2.5454, Validation loss 0.0868
Epoch 7000, Training loss 2.5454, Validation loss 0.0881
Epoch 7500, Training loss 2.5

tensor([  2.3250,   0.2942, -10.1420], requires_grad=True)